# Getting results from molQTL analyses in the Athero-Express

This notebook extracts per-variant and per-gene results from a _cis_-acting and _trans_-acting eQTL and mQTL analysis of expression and DNA methylation in carotid plaques from the Athero-Express Biobank Study.


## Import necessary libraries

In [4]:
# Function to check for installation of required packages
def check_install_package(package_name):
    try:
        importlib.import_module(package_name)
    except ImportError:
        print(f'{package_name} is not installed. Installing it now...')
        subprocess.check_call(['pip', 'install', package_name])

import os
import glob
import importlib
from subprocess import check_output
import subprocess
import sys

# argument parsing
import argparse

# get date and time
import time
from datetime import datetime
from datetime import timedelta

# Pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool
check_install_package('pandas')
import pandas as pd

# pyarrow is supperior to loading parquet files
check_install_package('pyarrow')
import pyarrow as pa
import pyarrow.parquet as pq

# polars is a fast dataframe library
check_install_package('polars')
import polars as pl

# for statistical analysis
check_install_package('scipy')
from scipy import stats
import numpy as np

# scientific colourmaps
# https://www.fabiocrameri.ch/ws/media-library/8c4b111121ff448e843dfef9220bf613/readme_scientificcolourmaps.pdf
check_install_package('cmcrameri')
import cmcrameri as ccm
from cmcrameri import cm

# for plotting
check_install_package('matplotlib')
import matplotlib
import matplotlib.pyplot as plt
# if using a Jupyter notebook, include:
%matplotlib inline

# use Seaborn for visualisations
check_install_package('seaborn')
import seaborn as sns

# for handling GWAS data
import gwaslab as gl

Setting some functions and standard variables, as well creating some directories. 

In [5]:
# Create directories for the GWAS data and the reference data

# set some general defaults
TRAIT_OF_INTEREST = "HTRA1"  # Phenotype
PROJECTNAME = "HTRA1"

POPULATION = "EUR"

# general plotting directory
PLOTS_loc = "PLOTS"

# location to put molQTL results
molQTL_loc = "molQTL_results"

# Get today's date
today_date = datetime.now()

# Format the date as yyyymmdd
FORMATTED_TODAY = today_date.strftime("%Y%m%d")

# Check if the directory exists
if not os.path.exists(molQTL_loc):
    # If it doesn't exist, create it
    os.makedirs(molQTL_loc)

# Check if the directory exists
if not os.path.exists(PLOTS_loc):
    # If it doesn't exist, create it
    os.makedirs(PLOTS_loc)

# # regional association plots directory
# REG_PLOTS_loc = PLOTS_loc + "/Regional_Association_Plots"

# # Check if the directory exists
# if not os.path.exists(REG_PLOTS_loc):
#     # If it doesn't exist, create it
#     os.makedirs(REG_PLOTS_loc)

# Reference data directory
REF_loc = "/Users/slaan3/PLINK/references"
print(f'Checking contents of the reference directory:')
print(check_output(["ls", os.path.join(REF_loc)]).decode("utf8"))

# GWAS data directory
GWAS_loc = "/Users/slaan3/PLINK/_GWAS_Datasets"
print(f'Checking contents of the GWAS directory:')
print(check_output(["ls", os.path.join(GWAS_loc)]).decode("utf8"))

# molQTL data directory
NOM_CIS_EQTL_loc = "/Users/slaan3/git/CirculatoryHealth/molqtl/results/version1_aernas1_firstrun/nom_cis_eqtl"
print(f'Checking contents of the nominal cis-eQTL data directory:')
print(check_output(["ls", os.path.join(NOM_CIS_EQTL_loc)]).decode("utf8"))

PERM_CIS_MQTL_loc = "/Users/slaan3/git/CirculatoryHealth/molqtl/results/perm_cis_mqtl"
print(f'Checking contents of the permuted cis-mQTL data directory:')
print(check_output(["ls", os.path.join(PERM_CIS_MQTL_loc)]).decode("utf8"))

PERM_TRANS_EQTL_loc = "/Users/slaan3/git/CirculatoryHealth/molqtl/results/version1_aernas1_firstrun/perm_trans_eqtl"
print(f'Checking contents of the permuted trans-eQTL data directory:')
print(check_output(["ls", os.path.join(PERM_TRANS_EQTL_loc)]).decode("utf8"))

PERM_TRANS_MQTL_loc = (
    "/Users/slaan3/git/CirculatoryHealth/molqtl/results/perm_trans_mqtl"
)
print(f'Checking contents of the permuted trans-eQTL data directory:')
print(check_output(["ls", os.path.join(PERM_TRANS_MQTL_loc)]).decode("utf8"))

Checking contents of the reference directory:
1000G
HRC_r1_1_2016
HRCr11_1000Gp3v5
dbSNP
tcga

Checking contents of the GWAS directory:
_AAAgen
_BloodPressure
_CARDIoGRAM
_CHARGE_CAC
_CHARGE_cIMT_PlaquePresence
_DIAGRAM
_IMPROVE_OLINK
_ISGC
_LIPIDS
_PGC
_SCALLOP
_SMCs
_SiGN
_VERTIGO
_cojo
_magma

Checking contents of the nominal cis-eQTL data directory:
README.md
tensorqtl_cis_nominal_chr01.cis_qtl_pairs.chr01.parquet
tensorqtl_cis_nominal_chr02.cis_qtl_pairs.chr02.parquet
tensorqtl_cis_nominal_chr03.cis_qtl_pairs.chr03.parquet
tensorqtl_cis_nominal_chr04.cis_qtl_pairs.chr04.parquet
tensorqtl_cis_nominal_chr05.cis_qtl_pairs.chr05.parquet
tensorqtl_cis_nominal_chr06.cis_qtl_pairs.chr06.parquet
tensorqtl_cis_nominal_chr07.cis_qtl_pairs.chr07.parquet
tensorqtl_cis_nominal_chr08.cis_qtl_pairs.chr08.parquet
tensorqtl_cis_nominal_chr09.cis_qtl_pairs.chr09.parquet
tensorqtl_cis_nominal_chr10.cis_qtl_pairs.chr10.parquet
tensorqtl_cis_nominal_chr11.cis_qtl_pairs.chr11.parquet
tensorqtl_cis_nomi

In [6]:
# Example function to get data from a molQTL file
def molqtl_merge_and_export(target_variants, sumstats, left_col, right_col, sort_column, output_csv):
    print(f'Merging target variants with nominal cis-eQTLs.')

    # Perform the join operation
    temp = target_variants.join(
        sumstats, left_on=left_col, right_on=right_col, how="inner"
    )

    print(
        f'> Sorting the DataFrame by column "{sort_column}" in descending order.')
    # Sort the DataFrame by specified column in descending order
    result = temp.sort(sort_column)
    del temp

    print(f'> Showing the first 5 rows of the DataFrame.')
    # Display the sorted DataFrame in descending order
    print(result)

    print(f'> Exporting the Polars DataFrame to a CSV file.')
    # Export the Polars DataFrame to a CSV file
    result.write_csv(output_csv)

# Example usage

# molqtl_merge_and_export(target_variants, sumstats_nom_cis_eqtl, "VariantID", "VariantID",
#                         "pval_nominal", os.path.join(molQTL_loc, "pgc_target_variants_nom_cis_eqtl.csv"))

# Targets

Here we load the list of variants of interest for this project.

In [7]:
# Get the list of targets
# polars.read_excel(
# source: str | BytesIO | Path | BinaryIO | bytes,
# *,
# sheet_id: None = None,
# sheet_name: str,
# engine: Literal['xlsx2csv', 'openpyxl', 'pyxlsb'] | None = None,
# xlsx2csv_options: dict[str, Any] | None = None,
# read_csv_options: dict[str, Any] | None = None,
# schema_overrides: SchemaDict | None = None,
# raise_if_empty: bool = True,
# )

target_variants = pl.read_excel(
    source=os.path.join("targets/targets.xlsx"), sheet_name="Variants"
)

In [8]:
target_variants

RSID,ALTID,VariantID,Chr,BP,VariantType,Comments,A1,A2,Link
str,str,str,i64,i64,str,str,str,str,str
"""rs2672592""","""---""","""10:124230750""",10,124230750,"""single nucleot…","""multi-allelic""","""G""","""T (A)""","""https://www.nc…"
null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null


# cis-eQTL (nominal)

Here we load the nominal _cis_-acting eQTL data.

In [9]:
# read in data
# https://stackoverflow.com/questions/33813815/how-to-read-a-parquet-file-into-pandas-dataframe

import polars as pl

# annotated cis-eQTLs
sumstats_nom_cis_eqtl = pl.read_parquet(
    source=os.path.join(
        NOM_CIS_EQTL_loc, "tensorqtl_nominal_cis_qtl_pairs.annot.parquet"
    )
)

In [10]:
sumstats_nom_cis_eqtl

EnsemblID,VariantID,tss_distance,CAF_eQTL,ma_samples,ma_count,pval_nominal,Beta,SE,chromosome,position,OtherAlleleA,CodedAlleleB,CAF,AltID,Source,AverageMaximumPosteriorCall,Info,AA_N,AB_N,BB_N,TotalN,MAF,MissingDataProportion,HWE_P
str,str,i32,f32,i32,i32,f64,f32,f32,i64,i64,str,str,f64,str,str,f64,f64,f64,f64,f64,i64,f64,f64,f64
"""ENSG0000018763…","""1:693731""",-240519,0.134185,153,168,0.881492,0.005953,0.039917,1,693731,"""A""","""G""",0.137241,"""rs12238997""","""HRCr11""",0.902046,0.624501,1637.01,451.103,35.569,2124,0.122957,0.000075,0.544841
"""ENSG0000018763…","""1:714596""",-219654,0.038339,48,48,0.834746,-0.015369,0.073635,1,714596,"""T""","""C""",0.0327213,"""rs149887893""","""HRCr11""",0.968278,0.577564,1987.73,135.495,0.677,2124,0.0322165,0.000024,0.272504
"""ENSG0000018763…","""1:715367""",-218883,0.038339,48,48,0.834746,-0.015369,0.073635,1,715367,"""A""","""G""",0.0324859,"""rs12184277""","""HRCr11""",0.975207,0.67196,1976.85,146.512,0.556,2124,0.0347528,0.000019,0.176551
"""ENSG0000018763…","""1:717485""",-216765,0.038339,48,48,0.834746,-0.015369,0.073635,1,717485,"""C""","""A""",0.0324859,"""rs12184279""","""HRCr11""",0.975183,0.670147,1978.35,145.041,0.517,2124,0.0343883,0.000022,0.174817
"""ENSG0000018763…","""1:720381""",-213869,0.038339,48,48,0.834746,-0.015369,0.073635,1,720381,"""G""","""T""",0.0327213,"""rs116801199""","""HRCr11""",0.974476,0.667536,1973.72,149.503,0.691,2124,0.0355205,0.00002,0.110452
"""ENSG0000018763…","""1:721290""",-212960,0.038339,48,48,0.834746,-0.015369,0.073635,1,721290,"""G""","""C""",0.0327213,"""rs12565286""","""HRCr11""",0.975168,0.677386,1972.67,150.564,0.687,2124,0.0357682,0.000017,0.110447
"""ENSG0000018763…","""1:726794""",-207456,0.038339,48,48,0.834746,-0.015369,0.073635,1,726794,"""C""","""G""",0.0324859,"""rs28454925""","""HRCr11""",0.977817,0.70519,1977.3,146.099,0.547,2124,0.0346509,0.000013,0.176493
"""ENSG0000018763…","""1:729632""",-204618,0.038339,48,48,0.834746,-0.015369,0.073635,1,729632,"""C""","""T""",0.0324859,"""rs116720794""","""HRCr11""",0.978581,0.715529,1976.76,146.752,0.454,2124,0.0347604,0.000008,0.176551
"""ENSG0000018763…","""1:729679""",-204571,0.829073,195,214,0.678049,0.015322,0.03689,1,729679,"""C""","""G""",0.829331,"""rs4951859""","""HRCr11""",0.936062,0.78861,58.204,600.893,1464.61,2124,0.16888,0.00007,0.757157


In [12]:
molqtl_merge_and_export(target_variants, sumstats_nom_cis_eqtl, "VariantID", "VariantID",
                        "pval_nominal",
                        os.path.join(molQTL_loc, FORMATTED_TODAY + "." + PROJECTNAME + ".nom_cis_eqtl.csv"))

Merging target variants with nominal cis-eQTLs.
> Sorting the DataFrame by column "pval_nominal" in descending order.
> Showing the first 5 rows of the DataFrame.
shape: (10, 34)
┌───────────┬───────┬──────────────┬─────┬───┬────────┬──────────┬──────────────────────┬──────────┐
│ RSID      ┆ ALTID ┆ VariantID    ┆ Chr ┆ … ┆ TotalN ┆ MAF      ┆ MissingDataProportio ┆ HWE_P    │
│ ---       ┆ ---   ┆ ---          ┆ --- ┆   ┆ ---    ┆ ---      ┆ n                    ┆ ---      │
│ str       ┆ str   ┆ str          ┆ i64 ┆   ┆ i64    ┆ f64      ┆ ---                  ┆ f64      │
│           ┆       ┆              ┆     ┆   ┆        ┆          ┆ f64                  ┆          │
╞═══════════╪═══════╪══════════════╪═════╪═══╪════════╪══════════╪══════════════════════╪══════════╡
│ rs2672592 ┆ ---   ┆ 10:124230750 ┆ 10  ┆ … ┆ 2124   ┆ 0.335975 ┆ 0.000006             ┆ 0.495824 │
│ rs2672592 ┆ ---   ┆ 10:124230750 ┆ 10  ┆ … ┆ 2124   ┆ 0.335975 ┆ 0.000006             ┆ 0.495824 │
│ rs2672592 ┆

In [13]:
# Clean up

del sumstats_nom_cis_eqtl

# cis-mQTL (permuted)

Here we load the nominal _cis_-acting mQTL data.

In [14]:
# read in data
import polars as pl

# Specify the file path to your data
file_path = os.path.join(PERM_CIS_MQTL_loc, "tensormqtl.perm_cis_mqtl.txt")

# Read the data into a Polars DataFrame
sumstats_perm_cis_mqtl = pl.read_csv(
    file_path, has_header=True, separator="\t", ignore_errors=True
)

In [15]:
# Display the first few rows of the DataFrame to verify the data loading
sumstats_perm_cis_mqtl

phenotype_id,num_var,beta_shape1,beta_shape2,true_df,pval_true_df,variant_id,tss_distance,ma_samples,ma_count,af,pval_nominal,slope,slope_se,pval_perm,pval_beta,qval,pval_nominal_threshold
str,i64,f64,f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64
"""cg21870274""",309,1.01037,15.0882,351.533,0.0495503,"""1:752307""",682715,15,15,0.0170068,0.036876,-0.406806,0.194254,0.531247,0.530703,0.468602,0.001232
"""cg08258224""",731,1.03658,40.9219,342.718,0.101636,"""1:1636400""",836316,37,38,0.0430839,0.0780652,-0.160865,0.0910595,0.984102,0.986523,0.605861,0.000509
"""cg18147296""",750,1.02594,43.8044,350.307,0.000046,"""1:771410""",-41130,130,141,0.840136,0.000014,-0.195278,0.0444456,0.0017,0.001693,0.006203,0.000454
"""cg13938959""",787,1.00503,41.991,346.635,0.0160055,"""1:800193""",-33991,7,7,0.007937,0.009935,0.568674,0.219518,0.49525,0.48971,0.45302,0.000433
"""cg12445832""",787,1.01013,42.2714,344.888,0.003059,"""1:1706886""",872590,44,46,0.0521542,0.001483,-0.25942,0.08106,0.118388,0.118428,0.211842,0.000439
"""cg23999112""",787,1.03227,42.5622,342.63,0.0111282,"""1:810286""",-24071,114,124,0.14059,0.006277,0.168039,0.0611592,0.364464,0.363523,0.395055,0.00048
"""cg08128007""",799,1.01323,45.7034,353.328,0.025363,"""1:1037047""",197611,177,207,0.234694,0.0177802,0.174208,0.0731943,0.686431,0.685759,0.522438,0.000412
"""cg23733394""",799,1.02275,42.9169,345.861,0.000206,"""1:753405""",-86348,108,118,0.866213,0.00007,-0.393622,0.0979262,0.006899,0.007824,0.0248428,0.000457
"""cg13371836""",800,0.994356,47.1897,362.606,0.0186669,"""1:1830151""",989876,125,141,0.159864,0.0138438,-0.193078,0.0780974,0.588241,0.591562,0.491116,0.000367


In [16]:
molqtl_merge_and_export(target_variants, sumstats_perm_cis_mqtl, "VariantID", "variant_id",
                        "pval_nominal",
                        os.path.join(molQTL_loc, FORMATTED_TODAY + "." + PROJECTNAME + ".perm_cis_eqtl.csv"))

Merging target variants with nominal cis-eQTLs.
> Sorting the DataFrame by column "pval_nominal" in descending order.
> Showing the first 5 rows of the DataFrame.
shape: (0, 27)
┌──────┬───────┬───────────┬─────┬───┬───────────┬───────────┬──────┬────────────────────────┐
│ RSID ┆ ALTID ┆ VariantID ┆ Chr ┆ … ┆ pval_perm ┆ pval_beta ┆ qval ┆ pval_nominal_threshold │
│ ---  ┆ ---   ┆ ---       ┆ --- ┆   ┆ ---       ┆ ---       ┆ ---  ┆ ---                    │
│ str  ┆ str   ┆ str       ┆ i64 ┆   ┆ f64       ┆ f64       ┆ f64  ┆ f64                    │
╞══════╪═══════╪═══════════╪═════╪═══╪═══════════╪═══════════╪══════╪════════════════════════╡
└──────┴───────┴───────────┴─────┴───┴───────────┴───────────┴──────┴────────────────────────┘
> Exporting the Polars DataFrame to a CSV file.


In [17]:
del sumstats_perm_cis_mqtl

# trans-eQTL (permuted)

Here we load the nominal _trans_-acting eQTL data.

In [18]:
# read in data
import polars as pl

# Specify the file path to your data
file_path = os.path.join(
    PERM_TRANS_EQTL_loc, "tensorqtl_trans_full.trans_qtl_pairs.parquet"
)

# Read the data into a Polars DataFrame
sumstats_perm_trans_eqtl = pl.read_parquet(file_path)

In [19]:
# Display the first few rows of the DataFrame to verify the data loading
sumstats_perm_trans_eqtl

variant_id,phenotype_id,pval,b,b_se,af,__index_level_0__
str,str,f64,f32,f32,f32,i64
"""1:730087""","""ENSG0000013140…",0.000009,-0.419756,0.093878,0.010383,0
"""1:752307""","""ENSG0000010803…",0.000006,-0.308768,0.067392,0.019169,1
"""1:752593""","""ENSG0000010803…",0.000006,-0.308768,0.067392,0.019169,2
"""1:752617""","""ENSG0000010803…",0.000006,-0.308768,0.067392,0.019169,3
"""1:754121""","""ENSG0000010803…",0.000006,-0.308768,0.067392,0.019169,4
"""1:754163""","""ENSG0000010803…",0.000006,-0.308768,0.067392,0.019169,5
"""1:754671""","""ENSG0000010803…",0.000006,-0.308768,0.067392,0.019169,6
"""1:759177""","""ENSG0000010803…",0.000006,-0.308768,0.067392,0.019169,7
"""1:759970""","""ENSG0000010803…",0.000006,-0.308768,0.067392,0.019169,8


In [20]:
molqtl_merge_and_export(target_variants, sumstats_perm_trans_eqtl, "VariantID", "variant_id",
                        "pval",
                        os.path.join(molQTL_loc, FORMATTED_TODAY + "." + PROJECTNAME + ".perm_trans_eqtl.csv"))

Merging target variants with nominal cis-eQTLs.
> Sorting the DataFrame by column "pval" in descending order.
> Showing the first 5 rows of the DataFrame.
shape: (0, 16)
┌──────┬───────┬───────────┬─────┬───┬─────┬──────┬─────┬───────────────────┐
│ RSID ┆ ALTID ┆ VariantID ┆ Chr ┆ … ┆ b   ┆ b_se ┆ af  ┆ __index_level_0__ │
│ ---  ┆ ---   ┆ ---       ┆ --- ┆   ┆ --- ┆ ---  ┆ --- ┆ ---               │
│ str  ┆ str   ┆ str       ┆ i64 ┆   ┆ f32 ┆ f32  ┆ f32 ┆ i64               │
╞══════╪═══════╪═══════════╪═════╪═══╪═════╪══════╪═════╪═══════════════════╡
└──────┴───────┴───────────┴─────┴───┴─────┴──────┴─────┴───────────────────┘
> Exporting the Polars DataFrame to a CSV file.


In [21]:
del sumstats_perm_trans_eqtl

# trans-mQTL (permuted)

Here we load the nominal _trans_-acting mQTL data.

In [22]:
# read in data
import polars as pl

# Specify the file path to your data
file_path = os.path.join(
    PERM_TRANS_MQTL_loc, "tensormqtl_perm_trans_qtl_pairs.annot.parquet"
)

# Read the data into a Polars DataFrame
sumstats_perm_trans_mqtl = pl.read_parquet(file_path)

In [23]:
# Display the first few rows of the DataFrame to verify the data loading
sumstats_perm_trans_mqtl

VariantID,CpG,pval_perm,Beta,SE,CAF_mQTL,__index_level_0__,chromosome,position,OtherAlleleA,CodedAlleleB,CAF,AltID,Source,AverageMaximumPosteriorCall,Info,AA_N,AB_N,BB_N,TotalN,MAF,MissingDataProportion,HWE_P
str,str,f64,f32,f32,f32,i64,i64,i64,str,str,f64,str,str,f64,f64,f64,f64,f64,i64,f64,f64,f64
"""1:693731""","""cg23922040""",0.000007,-0.356294,0.078441,0.126984,0,1,693731,"""A""","""G""",0.137241,"""rs12238997""","""HRCr11""",0.902046,0.624501,1637.01,451.103,35.569,2124,0.122957,0.000075,0.544841
"""1:714596""","""cg26079250""",4.7736e-7,0.601047,0.117393,0.038549,0,1,714596,"""T""","""C""",0.0327213,"""rs149887893""","""HRCr11""",0.968278,0.577564,1987.73,135.495,0.677,2124,0.0322165,0.000024,0.272504
"""1:715367""","""cg26079250""",4.7736e-7,0.601047,0.117393,0.038549,1,1,715367,"""A""","""G""",0.0324859,"""rs12184277""","""HRCr11""",0.975207,0.67196,1976.85,146.512,0.556,2124,0.0347528,0.000019,0.176551
"""1:717485""","""cg26079250""",4.7736e-7,0.601047,0.117393,0.038549,2,1,717485,"""C""","""A""",0.0324859,"""rs12184279""","""HRCr11""",0.975183,0.670147,1978.35,145.041,0.517,2124,0.0343883,0.000022,0.174817
"""1:720381""","""cg26079250""",4.7736e-7,0.601047,0.117393,0.038549,3,1,720381,"""G""","""T""",0.0327213,"""rs116801199""","""HRCr11""",0.974476,0.667536,1973.72,149.503,0.691,2124,0.0355205,0.00002,0.110452
"""1:721290""","""cg26079250""",4.7736e-7,0.601047,0.117393,0.038549,4,1,721290,"""G""","""C""",0.0327213,"""rs12565286""","""HRCr11""",0.975168,0.677386,1972.67,150.564,0.687,2124,0.0357682,0.000017,0.110447
"""1:726794""","""cg26079250""",4.7736e-7,0.601047,0.117393,0.038549,5,1,726794,"""C""","""G""",0.0324859,"""rs28454925""","""HRCr11""",0.977817,0.70519,1977.3,146.099,0.547,2124,0.0346509,0.000013,0.176493
"""1:729632""","""cg26079250""",4.7736e-7,0.601047,0.117393,0.038549,6,1,729632,"""C""","""T""",0.0324859,"""rs116720794""","""HRCr11""",0.978581,0.715529,1976.76,146.752,0.454,2124,0.0347604,0.000008,0.176551
"""1:729679""","""cg23922040""",0.000003,0.343842,0.072648,0.835601,1,1,729679,"""C""","""G""",0.829331,"""rs4951859""","""HRCr11""",0.936062,0.78861,58.204,600.893,1464.61,2124,0.16888,0.00007,0.757157


In [24]:
molqtl_merge_and_export(target_variants, sumstats_perm_trans_mqtl, "VariantID", "VariantID",
                        "pval_perm",
                        os.path.join(molQTL_loc, FORMATTED_TODAY + "." + PROJECTNAME + ".perm_trans_mqtl.csv"))

Merging target variants with nominal cis-eQTLs.
> Sorting the DataFrame by column "pval_perm" in descending order.
> Showing the first 5 rows of the DataFrame.
shape: (4, 32)
┌───────────┬───────┬──────────────┬─────┬───┬────────┬──────────┬──────────────────────┬──────────┐
│ RSID      ┆ ALTID ┆ VariantID    ┆ Chr ┆ … ┆ TotalN ┆ MAF      ┆ MissingDataProportio ┆ HWE_P    │
│ ---       ┆ ---   ┆ ---          ┆ --- ┆   ┆ ---    ┆ ---      ┆ n                    ┆ ---      │
│ str       ┆ str   ┆ str          ┆ i64 ┆   ┆ i64    ┆ f64      ┆ ---                  ┆ f64      │
│           ┆       ┆              ┆     ┆   ┆        ┆          ┆ f64                  ┆          │
╞═══════════╪═══════╪══════════════╪═════╪═══╪════════╪══════════╪══════════════════════╪══════════╡
│ rs2672592 ┆ ---   ┆ 10:124230750 ┆ 10  ┆ … ┆ 2124   ┆ 0.335975 ┆ 0.000006             ┆ 0.495824 │
│ rs2672592 ┆ ---   ┆ 10:124230750 ┆ 10  ┆ … ┆ 2124   ┆ 0.335975 ┆ 0.000006             ┆ 0.495824 │
│ rs2672592 ┆ ---

In [ ]:
del sumstats_perm_trans_mqtl